In [ ]:
import os
import cv2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from lxml import etree
from tqdm import tqdm

# Custom class dictionary
class_dict = {0: 'unclassified'}  # Class IDs start from 0 in Detectron2

# Load config from a config file
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/retinanet_R_101_FPN_3x.yaml'))
cfg.MODEL.WEIGHTS = '/home/gpu1/Documents/Projects/Mandar/train-object-detector-detectron2/output/model_final.pth'
cfg.MODEL.DEVICE = 'cuda'

# Create predictor instance
predictor = DefaultPredictor(cfg)

input_directory = input('Input directory: ')
output_directory = input('Output directory: ')

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get a list of image files
image_files = [f for f in os.listdir(input_directory) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# Create a tqdm progress bar
with tqdm(total=len(image_files), unit='image') as pbar:
    for image_file in image_files:
        image_path = os.path.join(input_directory, image_file)
        image = cv2.imread(image_path)

        # Perform prediction
        outputs = predictor(image)
        threshold = 0.5

        # Check if there are any detections with scores above the threshold
        has_detections = any(score > threshold for score in outputs["instances"].scores)

        if has_detections:
            # Create an XML annotation for the image with detections
            annotation = etree.Element("annotation")
            folder = etree.SubElement(annotation, "folder")
            folder.text = os.path.basename(input_directory)
            filename = etree.SubElement(annotation, "filename")
            filename.text = image_file
            source = etree.SubElement(annotation, "source")
            database = etree.SubElement(source, "database")
            database.text = "Unknown"
            annotation_element = etree.SubElement(source, "annotation")
            annotation_element.text = "Unknown"
            image_source = etree.SubElement(source, "image")
            image_source.text = "Unknown"
            size = etree.SubElement(annotation, "size")
            width = etree.SubElement(size, "width")
            height = etree.SubElement(size, "height")
            height.text = str(image.shape[0])
            width.text = str(image.shape[1])
            segmented = etree.SubElement(annotation, "segmented")
            segmented.text = "0"

            # Loop through the detected objects and add them to the XML
            for j, bbox in enumerate(outputs["instances"].pred_boxes.tensor):
                x1, y1, x2, y2 = [int(i) for i in bbox.tolist()]
                score = outputs["instances"].scores[j]
                pred = outputs["instances"].pred_classes[j]

                if score > threshold:
                    object_elem = etree.SubElement(annotation, "object")
                    name = etree.SubElement(object_elem, "name")
                    name.text = class_dict[int(pred)]
                    truncated = etree.SubElement(object_elem, "truncated")
                    truncated.text = "0"
                    occluded = etree.SubElement(object_elem, "occluded")
                    occluded.text = "0"
                    difficult = etree.SubElement(object_elem, "difficult")
                    difficult.text = "0"
                    bndbox = etree.SubElement(object_elem, "bndbox")
                    xmin = etree.SubElement(bndbox, "xmin")
                    ymin = etree.SubElement(bndbox, "ymin")
                    xmax = etree.SubElement(bndbox, "xmax")
                    ymax = etree.SubElement(bndbox, "ymax")
                    xmin.text = str(x1)
                    ymin.text = str(y1)
                    xmax.text = str(x2)
                    ymax.text = str(y2)

            # Save the annotation as an XML file
            annotation_file = os.path.splitext(image_file)[0] + '.xml'
            annotation_path = os.path.join(output_directory, annotation_file)
            with open(annotation_path, 'wb') as xml_file:
                xml_file.write(etree.tostring(annotation, pretty_print=True))

            # Copy the image to the output directory
            output_image_path = os.path.join(output_directory, image_file)
            cv2.imwrite(output_image_path, image)

        # Update the progress bar
        pbar.update(1)
